<a href="https://colab.research.google.com/github/tnakagawa/ipynb/blob/master/Reducing_Bitcoin_Transaction_Sizes_with_x_only_Pubkeys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# x座標だけの公開鍵によるBitcoinのTransactionの削減

[Qiita](https://qiita.com/tnakagawa/private/52666def36afdeabd05f)で投稿した内容のソース検証用です。

In [0]:
#@title secp256k1実装（ダブルクリックで閲覧可）
class S256P:
    p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
    n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

    def __init__(self, x, y):
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        self.x = self.x % self.p
        self.y = self.y % self.p
        if pow(self.y, 2, self.p) != (pow(self.x, 3, self.p) + 7) % self.p:
            raise ValueError(
                'The x and y are not on curve.{} {}'.format(x.num, y.num))

    def __add__(self, other):
        if self.x == None:
            return other.__class__(other.x, other.y)
        if other.x == None:
            return self.__class__(self.x, self.y)
        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None)
        if self.x != other.x:
            s = ((self.y - other.y)
                 * pow(self.x - other.x, self.p - 2, self.p)) % self.p
            x = (pow(s, 2, self.p) - self.x - other.x) % self.p
            y = (s * (self.x - x) - self.y) % self.p
            return self.__class__(x, y)
        if self == other and self.y == 0 * self.x:
            return self.__class__(None, None)
        if self == other:
            s = ((3 * pow(self.x, 2, self.p))
                 * pow(2 * self.y, self.p - 2, self.p)) % self.p
            x = (pow(s, 2, self.p) - 2 * self.x) % self.p
            y = (s * (self.x - x) - self.y) % self.p
            return self.__class__(x, y)

    def __rmul__(self, coefficient):
        if self.x is None:
            return self.__class__(None, None)
        coef = coefficient % self.n
        current = self.__class__(self.x, self.y)
        result = self.__class__(None, None)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result

    def X(self):
        return self.x.to_bytes(32, 'big')

    def compress(self):
        if self.y % 2 == 0:
            return b'\x02' + self.x.to_bytes(32, 'big')
        else:
            return b'\x03' + self.x.to_bytes(32, 'big')

    def uncompress(self):
        return b'\x04' + self.x.to_bytes(32, 'big') + self.y.to_bytes(32, 'big')


def parse(compress):
    if len(compress) != 33:
        raise ValueError('illegal length: {}'.format(len(compress)))
    even = True
    if compress[0] == 3:
        even = False
    x = int.from_bytes(compress[1:33], 'big')
    y2 = (pow(x, 3, S256P.p) + 7) % S256P.p
    y = pow(y2, (S256P.p + 1) >> 2, S256P.p)
    if even:
        if y % 2 == 1:
            y = S256P.p - y
    else:
        if y % 2 == 0:
            y = S256P.p - y
    return S256P(x, y)


G = S256P(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
          0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)


## Schnorrの署名と検証

Schnorrの署名(SchnorrSign)と検証(SchnorrVerify)を実装します。

署名は、署名者が公開鍵を知っているので、公開鍵のy座標が偶数か奇数かで、署名値の計算方法を変えます。

検証は、公開鍵のx座標しかしらないので、偶数`0x02`であるとして公開鍵をパースして検証します。

In [0]:
import hashlib
import random

def SchnorrSign(k, R, P, m, d):
    h = int.from_bytes(hashlib.sha256(R.compress() + P.X() + m).digest(), 'big')
    if P.y % 2 == 0:
        return (k + h * d) % S256P.n
    else:
        return (k - h * d) % S256P.n


def SchnorrVerify(s, Ra, Px, m):
    sG = s * G
    h = int.from_bytes(hashlib.sha256(Ra + Px + m).digest(), 'big')
    P = parse(b'\x02' + Px)
    R = parse(Ra)
    Q = R + h * P
    return (sG.x == Q.x and sG.y == Q.y)


## 確認

実際に、Schnoorの署名と検証を行ってみます。

まずは、公開鍵のy座標が偶数`0x02`である場合について署名と検証を行います。

次に、秘密鍵の負値にすることで公開鍵のy座標が奇数`0x03`となります。

この場合について署名と検証を行います。

In [0]:
#@title 公開鍵が0x02で始まる場合
m = b'message'

d = random.randint(1, S256P.n - 1)
P = d * G
if P.y % 2 == 1:
    d = -d
    P = d * G

k = random.randint(1, S256P.n - 1)
R = k * G

print('-----  SchnorrSign  -----')
print('random point key : **********')
print('  random point R :', R.compress().hex())
print('\033[32m'+' compress Pubkey :'+'\033[0m',
      '\033[32m'+P.compress().hex()+'\033[0m')
print('         message :', m.hex())
print('     private key : **********')
s = SchnorrSign(k, R, P, m, d)
print('     signature s :', s.to_bytes(32, 'big').hex())

Px = P.X()
Ra = R.compress()

print('----- SchnorrVerify -----')
print('     signature s :', s.to_bytes(32, 'big').hex())
print('     signature R :', Ra.hex())
print('\033[34m'+'   x-only Pubkey :'+'\033[0m',
      '\033[34m'+P.X().hex()+'\033[0m')
print('         message :', m.hex())
result = SchnorrVerify(s, Ra, Px, m)
print('\033[31m'+'result of verify :'+'\033[0m',
      '\033[31m'+str(result)+'\033[0m')

In [0]:
#@title 公開鍵が0x03で始まる場合
d = -d
P = d * G
k = random.randint(1, S256P.n - 1)
R = k * G

print('-----  SchnorrSign  -----')
print('random point key : **********')
print('  random point R :', R.compress().hex())
print('\033[32m'+' compress Pubkey :'+'\033[0m',
      '\033[32m'+P.compress().hex()+'\033[0m')
print('         message :', m.hex())
print('     private key : **********')
s = SchnorrSign(k, R, P, m, d)
print('     signature s :', s.to_bytes(32, 'big').hex())

Px = P.X()
Ra = R.compress()

print('----- SchnorrVerify -----')
print('     signature s :', s.to_bytes(32, 'big').hex())
print('     signature R :', Ra.hex())
print('\033[34m'+'   x-only Pubkey :'+'\033[0m',
      '\033[34m'+P.X().hex()+'\033[0m')
print('         message :', m.hex())
result = SchnorrVerify(s, Ra, Px, m)
print('\033[31m'+'result of verify :'+'\033[0m',
      '\033[31m'+str(result)+'\033[0m')


# 結果

どちらも、署名側の<font color=green>公開鍵（`compress Pubkey`）</font>は先頭が`0x02`と`0x03`で異なりますが、検証側の<font color=blue>x座標だけの公開鍵（`x-only Pubkey`）</font>は同じです。

検証結果はどちらも、<font color=red>True</font>となっているはずです。